In [ ]:
pip install langchain langchain_community langchain_groq sentence_transformers faiss-cpu langchain_google_genai chromadb

  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_groq-0.3.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 13.4 MB/s eta 0:00:

In [ ]:
from google.colab import userdata
import os
GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

GROQ_API_KEY = userdata.get("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-12-3419084168.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/")

docs = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)

splitted_docs = splitter.split_documents(docs)

In [ ]:
import os
os.environ["CHROMA_TELEMETRY"] = "False"

import logging
logging.getLogger("chromadb").setLevel(logging.CRITICAL)

In [ ]:
from langchain_community.vectorstores import Chroma
vector_store = Chroma.from_documents(
    documents= splitted_docs,
    collection_name="prompt-engineering",
    embedding=embeddings
)

In [ ]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 4})

In [ ]:
from langchain.prompts import ChatPromptTemplate
template = """For the Given Question try to generate a hypothetical answer\
Only generate the answer and nothing else:
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
query = prompt.format(question = 'What are the different Chain of Thought(COT) Prompting?')

hypothetical_answer = llm.invoke(query).content
print(hypothetical_answer)

Chain-of-Thought prompting involves guiding a language model through a series of logical steps to arrive at a solution, mimicking how humans think through problems. 

There are several variations of COT prompting:

* **Step-by-Step:** The prompt explicitly breaks down the problem into a sequence of smaller steps, requiring the model to reason through each one.

* **Self-Guided:**  The prompt encourages the model to identify the necessary steps itself, often by asking "What would be the next step?"

* **Reasoning Chains:** The prompt provides a starting point and then guides the model to generate a chain of logical deductions, leading to the final answer.

* **Hypothesis Generation:** The prompt encourages the model to generate multiple hypotheses and then evaluate them step-by-step to arrive at the most likely solution.


Each variation has its own strengths and weaknesses, depending on the complexity of the task and the capabilities of the language model. 




In [ ]:
similiar_docs = retriever.get_relevant_documents(hypothetical_answer)

for doc in similiar_docs:
  print(doc.page_content)
  print()

Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using

[9] Diao et al. “Active Prompting with Chain-of-Thought for Large Language Models.” arXiv preprint arXiv:2302.12246 (2023).
[10] Zelikman et al. “STaR: Bootstrapping Reasoning With Reasoning.” arXiv preprint arXiv:2203.14465 (2022).

[17] Zhang et al. “Automatic chain of thought prompting in large language models.” arXiv preprint arXiv:2210.03493 (2022).
[18] Shum et al. “Automatic Prompt Augmentation and Selection with Chain-of-Thought from Labeled Data.” arXiv preprint arXiv:2302.12822 (2023).

[7] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022
[8] Wang et al. “Self-Consistency Improves Chain of Thought Reasoning in Language Models.” ICLR 20

In [ ]:
similiar_docs_naive = retriever.get_relevant_documents("What are the different Chain of Thought(COT) Prompting?")
for doc in similiar_docs_naive:
  print(doc.page_content)
  print()

Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using

Chain-of-Thought (CoT)#

Types of CoT prompts#
Two main types of CoT prompting:

Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, essentially creating a tree structure. The search process can be BFS or DFS while each



In [ ]:
template = """Answer the following Questions in the detail based on this context:
{context}
Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
formatted_docs = format_docs(similiar_docs)
formatted_docs

'Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using\n\n[9] Diao et al. “Active Prompting with Chain-of-Thought for Large Language Models.” arXiv preprint arXiv:2302.12246 (2023).\n[10] Zelikman et al. “STaR: Bootstrapping Reasoning With Reasoning.” arXiv preprint arXiv:2203.14465 (2022).\n\n[17] Zhang et al. “Automatic chain of thought prompting in large language models.” arXiv preprint arXiv:2210.03493 (2022).\n[18] Shum et al. “Automatic Prompt Augmentation and Selection with Chain-of-Thought from Labeled Data.” arXiv preprint arXiv:2302.12822 (2023).\n\n[7] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022\n[8] Wang et al. “Self-Consistency Improves Chain of Thought Reasoning in Language Models

In [ ]:
formatted_docs_navie = format_docs(similiar_docs_naive)
formatted_docs_navie

'Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using\n\nChain-of-Thought (CoT)#\n\nTypes of CoT prompts#\nTwo main types of CoT prompting:\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, essentially creating a tree structure. The search process can be BFS or DFS while each'

In [ ]:
query_prompt = prompt.format(context = formatted_docs,
                             question = "What are the different Chain of Thought(COT) Prompting?")

print(query_prompt)

Human: Answer the following Questions in the detail based on this context:
Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using

[9] Diao et al. “Active Prompting with Chain-of-Thought for Large Language Models.” arXiv preprint arXiv:2302.12246 (2023).
[10] Zelikman et al. “STaR: Bootstrapping Reasoning With Reasoning.” arXiv preprint arXiv:2203.14465 (2022).

[17] Zhang et al. “Automatic chain of thought prompting in large language models.” arXiv preprint arXiv:2210.03493 (2022).
[18] Shum et al. “Automatic Prompt Augmentation and Selection with Chain-of-Thought from Labeled Data.” arXiv preprint arXiv:2302.12822 (2023).

[7] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022
[8] Wang et al. “Self-C

In [ ]:
response = llm.invoke(query_prompt)
print(response.content)

The provided text describes Chain-of-Thought (CoT) prompting as a technique where a sequence of short sentences, called reasoning chains or rationales, is used to guide a language model through a problem-solving process.  

However, it doesn't delve into *specific* types or variations of CoT prompting. 

To answer your question about different types of CoT prompting, we need to look beyond the given context.  Here are some common variations:

**1. Manual CoT Prompting:**

* This involves a human carefully crafting the reasoning chain, step-by-step, to guide the model towards the desired answer.
* It requires domain expertise and understanding of the problem-solving process.

**2. Automatic CoT Prompting:**

* This uses algorithms and techniques to automatically generate the reasoning chain.
* Examples include:
    * **Template-based prompting:** Using predefined templates with slots for filling in specific information.
    * **Instruction tuning:** Training models on datasets of CoT ex

In [ ]:

query_prompt_naive = prompt.format(context = formatted_docs,
                             question = "What are the different Chain of Thought(COT) Prompting?")

print(query_prompt_naive)

Human: Answer the following Questions in the detail based on this context:
Chain-of-thought (CoT) prompting (Wei et al. 2022) generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using

[9] Diao et al. “Active Prompting with Chain-of-Thought for Large Language Models.” arXiv preprint arXiv:2302.12246 (2023).
[10] Zelikman et al. “STaR: Bootstrapping Reasoning With Reasoning.” arXiv preprint arXiv:2203.14465 (2022).

[17] Zhang et al. “Automatic chain of thought prompting in large language models.” arXiv preprint arXiv:2210.03493 (2022).
[18] Shum et al. “Automatic Prompt Augmentation and Selection with Chain-of-Thought from Labeled Data.” arXiv preprint arXiv:2302.12822 (2023).

[7] Wei et al. “Chain of thought prompting elicits reasoning in large language models.” NeurIPS 2022
[8] Wang et al. “Self-C

In [ ]:
response_naive = llm.invoke(query_prompt_naive)
print(response_naive.content)

The provided text describes Chain-of-Thought (CoT) prompting as a technique where a sequence of short sentences, called reasoning chains or rationales, guides a large language model (LLM) to arrive at an answer. 

However, it doesn't delve into *different types* of CoT prompting. 

To answer your question, we need to look beyond the given context. Here are some distinctions in CoT prompting approaches:

**1. Manually Crafted CoT Prompts:**

* These are designed by humans, explicitly outlining each step of the reasoning process. 
* Example:  For a question like "What is 2 + 2?", a CoT prompt might be:
    * "Let's figure this out step-by-step. 
    * First, we know that 2 + 1 = 3.
    * Second, 3 + 1 = 4.
    * Therefore, 2 + 2 = 4."

**2. Automatically Generated CoT Prompts:**

* Researchers are exploring methods to automatically generate CoT prompts based on:
    * **Dataset Analysis:** Identifying patterns in successful human reasoning for similar tasks.
    * **LLM Self-Supervision: